In [ ]:
# import packages
import itertools
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd
# import modin.pandas as pd # for faster parallelized processing
# from distributed import Client # this is required when using modin with dask

In [ ]:
# client = Client() # this is required when using modin with dask

In [ ]:
def pairwise_distance_vector(arr_coord):
    p_dsit_arr = pairwise_cdist(arr_coord)      
    # Get the indices of the upper-triangle elements of the matrix
    indices = np.triu_indices(p_dsit_arr.shape[0], k=1)
    # Extract the upper-triangle elements as a 1D vector
    pairwise_dist_vec = p_dsit_arr[indices]
    return pairwise_dist_vec

def pairwise_cdist(coord_lst_arr):
    return cdist(coord_lst_arr, coord_lst_arr)

In [ ]:
# import data
raw_data_dir = r"F:\Dropbox (UFL)\sleap_umap_tube_test_03312023\tube_test_analysis\all_matches_videos\\"
coord_filename = r"all_matches_videos_node_coordinates.csv"
df_data = pd.read_csv(raw_data_dir+coord_filename, index_col="Unnamed: 0")
# get all the identifiers in column headers
node_names = ['ear', 'nose', 'tail_base', 'thorax', 'front_foot', 'rear_foot']
track_names = ["track_0", "track_1"]
track_0_node_names = ["track_0__" + s for s in node_names]
track_1_node_names = ["track_1__" + s for s in node_names]
track_node_names = track_0_node_names + track_1_node_names
dist_pair_names = ['__dist_to__'.join(x) for x in list(itertools.combinations(track_node_names, 2))]
axis_names = ["x", "y"]
trials_lst = df_data['trial'].unique().tolist()
df_data

In [ ]:
# coordinate numpy array pairing x and y coordinates
arr_data = np.array(df_data.iloc[:,:-1])
arr_coord_data = arr_data.reshape(arr_data.shape[0], int(arr_data.shape[1]/2), 2)

# calcualte all pairwise distances
# not the msot efficient with a for loop here, but it gets the job done for this amount of data
pairwise_distance_lst = []
for i in range(len(arr_coord_data)): 
    pairwise_distance_lst.append(pairwise_distance_vector(arr_coord_data[i,:,:]))
pairwise_distance_arr = np.array(pairwise_distance_lst)
pairwise_distance_df = pd.DataFrame(pairwise_distance_arr)
pairwise_distance_df.columns = dist_pair_names
pairwise_distance_df['trial'] = df_data['trial']

pairwise_distance_df

In [ ]:
# save the dsitances to a csv
dist_filename = r"all_matches_videos_node_distances.csv"
pairwise_distance_df.to_csv(raw_data_dir+dist_filename)